# <font color='MediumBlue ' style="font-size:40px"><center><b>Problem set 2</b></center></font>

<font color='DarkViolet ' style="font-size:25px"><b>Problem 1 - Performance measures</b></font>

In this task you are invited to practice the calculation and interpretation of different performance measures. Before you start, please upload "weekly4.xlsx" into Google Colab.

a) Import data on simple weekly returns on selected stocks (they are stored in sheet 1 of the "weekly4.xlsx" file).

In [2]:
from google.colab import files
uploaded0 = files.upload()

Saving weekly4.xlsx to weekly4.xlsx


In [4]:
import pandas as pd
import datetime as dt
import numpy as np
from google.colab import files
uploaded1= files.upload()

Saving kvantitativne_formule.py to kvantitativne_formule.py


In [6]:
import kvantitativne_formule as qt
import scipy.stats as st
import scipy.optimize as sco
import numpy.linalg as LA

In [7]:
ret = pd.read_excel('weekly4.xlsx',index_col="Date")
ret

,MSFT,NKE,PEP,TSLA
Date,,,,
2020-01-06,0.017003,-0.010058,-0.008143,0.076332
2020-01-13,0.035079,0.035344,0.048815,0.065466
2020-01-20,-0.012405,-0.024207,0.011683,0.101116
2020-01-27,0.030963,-0.057799,-0.006317,0.141342
2020-02-03,0.077187,0.032086,0.023314,0.139648
...,...,...,...,...
2022-11-14,-0.024124,-0.006335,0.018254,-0.083950
2022-11-28,0.029972,0.057221,0.008545,0.063561
2022-12-12,-0.002979,-0.032226,-0.015189,-0.175498


b) Compute the <u>expected returns</u> using one of the 2 possible measures that we have seen in class (metric which you decide to use continue using in all further tasks which requires expected returns as inputs in this problem and the next). State in words the calculation method. Which stock is expected to perform the best and which is supposed to perform the worst according to the method of estimation that you have selected?

In [9]:
er = qt.expRet(ret,n=52,com=True)
er

MSFT    0.230438
NKE     0.159351
PEP     0.197626
TSLA    0.633493
dtype: float64

I have used annualized compounded sample returns method. According to this method, Tesla is expected to perform the best with return 63.35%, while worst performer in this case will be nike with return 15.94%

c) Estimate the level of risk associated with each of these 4 investments as annualized volatility. What is the riskiest and what is the least risky stock, according to this measure.  

In [11]:
vol = qt.annualize_vol(ret,n=52)
vol

MSFT    0.282926
NKE     0.368601
PEP     0.236546
TSLA    0.708004
dtype: float64

<font color='DodgerBlue'> Tesla stock is the riskiest, while pepsi have lowest risk </font>

d) What is the Sharpe ratio for each of these 4 investments? Which stock offers the best risk-return trade-off? According to FRED, average risk-free rate in this period was 0.6628% annually.

In [12]:
sharp = qt.sharpe(ret,52,rf=0.006628,com=True)
sharp

MSFT    0.791054
NKE     0.414332
PEP     0.807448
TSLA    0.885398
dtype: float64

<font color='Green'>According to Sharpe's ratio, i can conclude that these investments have higher risks than returns, while Sharpe's ratio si less than one. From this results Tesla's stock have the best risk/return trade-off.

e) In class we have mentioned another risk-adjusted measure of performance - Calmar's ratio. Compute it for all 4 stocks. How would we rank the investments according to the Calmar ratio?

$$CR=\frac{E(r)-r_f}{-MD}$$

In [14]:
def CR(xret,er,rf):
  xs=dict()
  for i in xret.columns:
    xs[i]=(er[i]-rf)/-((qt.drawdown(xret[i])['Drawdowns']).min())
  return xs

In [15]:
CR(ret,er,rf=0.006628)

{'MSFT': 0.6503674256631935,
 'NKE': 0.31320582264177327,
 'PEP': 0.9086339838898261,
 'TSLA': 0.8539028590382058}

Due the case of Calmar ratio the best investment is goning to be pepsi stocks. The bigger CR Ratio - the better risk/return trade-off off the investment.

<font color='Violet ' style="font-size:25px"><b>Problem 2 - Building (optimal) portfolio</b></font>

In this task we continue to work with the same data as in <font color='Violet '><b>Problem 1 </b></font>, but now we intend optimal portfolio of this 4 stocks.

a) Suppose you have decided to invest into the **equally-weighted portfolio** of the same 4 stocks as in <font color='Violet '><b>Problem 1 </b></font>. Calculate its annual expected return and volatility. State your results in words.

PS: When estimating portfolio expected return use the same statistic that you have used in previous problem.

In [16]:
w=np.array([0.25]*4)
w

array([0.25, 0.25, 0.25, 0.25])

In [19]:
ret_eq = qt.per(w,ret,52,com=True,er_assumed=None)
ret_eq #Let's find the returns for the equally weightened portfolio

0.30522707926547404

In [20]:
vol_eq = qt.pV(w,ret,52,vol=True)
vol_eq #we take the square root to find the standard deviation (with units being not squared)

0.3160412687458506

<font color='crimson'>Expected return for the portfolio is 30.52%, and the volatility is 31.6%.</font>


b) Find the MVP portfolio. Compare (via comments) it's risk, returns and weights with those of equally-weighted portfolio (analyzed in previous task).

In [22]:
result=sco.minimize(lambda w: qt.pV(w,ret,52),[0.25]*4,constraints=[dict(type='eq',fun=lambda w: sum(w)-1)])
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.047705868963458
       x: [ 3.538e-01  4.637e-02  6.766e-01 -7.679e-02]
     nit: 11
     jac: [ 9.547e-02  9.551e-02  9.550e-02  9.649e-02]
    nfev: 56
    njev: 11

In [23]:
np.sqrt(result['fun']) #we take the square root to find the standard deviation with units being not squared

0.2184167323339904

In [24]:
ret_mvp = qt.per(result.x,ret,52,com=True,er_assumed=None)
ret_mvp #Return for the MVP portfolio

0.17399134084525553

<font color='crimson'>The weights are different from the equally weighted portfolio with PEPSI having the highest weight. Note that TESLA is the one should sell short according to these computations. Our MVP portfolio is less risky (21.8% against 31.6% for the equally weighted portfolio). MVP portfolio has a smaller return compared to the equally weighted portfolio (17.4 % against 30.5%). </font>

c) Find portfolio which offers the same expected return as equally-weighted portfolio but with the smallest possible level of risk. Compare (via comments) risk (volatility) of that portfolio with the level of risk for portfolios discussed in a) and b).

In [26]:
result2=sco.minimize(lambda w: qt.pV(w,ret,52),[0.25]*4,constraints=[dict(type='eq',fun=lambda w:sum(w)-1),
                                                                     dict(type='eq',fun=lambda w: qt.per(w,ret,52,com=True,er_assumed=None)- ret_eq)])
result2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.08016296301671
       x: [ 3.473e-01 -1.798e-01  6.276e-01  2.049e-01]
     nit: 7
     jac: [ 1.233e-01  8.820e-02  1.071e-01  3.226e-01]
    nfev: 35
    njev: 7

In [27]:
ret_2 = qt.per(result2.x,ret,52,True,None)
ret_2

0.3052270792058216

In [28]:
np.sqrt(result2.fun)

0.2831306465515699

<font color='crimson'>Put your comment here:
New portfolio is less risky than the equally weighted portfolio (28.3% against 31.6% for the equally weighted portfolio), but more risky than MVP portfolio (21.8%).</font>

<font color='Teal' size=3><b>Bonus task</b></font><br>
d*) Assume that an investor wants to achieve return of 26.5% on the annual basis with the smallest possible level of maximal drawdawn. Suppose further that short selling is prohibited in the economy. What would be the optimal portfolio that you would recommend to him/her in such case? What is the MD of such portfolio?

In [31]:
result3=sco.minimize(lambda w: -qt.drawdown(ret@w)['Drawdowns'].min(),[0.25]*4,bounds=[(0,1)]*4,constraints=[dict(type='eq',fun=lambda w:sum(w)-1),
                                                                                                          dict(type='eq',fun=lambda w: qt.per(w,ret,52,True,None)-0.265)])
result3

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.19805254498254013
       x: [ 2.006e-01  1.027e-16  6.600e-01  1.395e-01]
     nit: 16
     jac: [ 1.942e-01  1.940e-01  1.035e-01  5.288e-01]
    nfev: 91
    njev: 16

In [33]:
ret

,MSFT,NKE,PEP,TSLA
Date,,,,
2020-01-06,0.017003,-0.010058,-0.008143,0.076332
2020-01-13,0.035079,0.035344,0.048815,0.065466
2020-01-20,-0.012405,-0.024207,0.011683,0.101116
2020-01-27,0.030963,-0.057799,-0.006317,0.141342
2020-02-03,0.077187,0.032086,0.023314,0.139648
...,...,...,...,...
2022-11-14,-0.024124,-0.006335,0.018254,-0.083950
2022-11-28,0.029972,0.057221,0.008545,0.063561
2022-12-12,-0.002979,-0.032226,-0.015189,-0.175498


In [32]:
[round(i,2) for i in list(result3.x)]

[0.2, 0.0, 0.66, 0.14]

<font color='crimson'>Put your comment here:</font>

<font color='crimson'>In order to calculate the maximal drawdown portfolio, i have to calculate the vector product of our portfolio weights and the expected returns of the assets, in order to get weekly returns for our portfolio: `ret@w`. This is done  since historical data is required for the computation of maximal drawdown.</font>

<font color='crimson'>According to the results of minimization, i would recommend to invest like that: `[0.2, 0.0, 0.66, 0.14]`. Maximal drawdown in that case is -0.198.
</font>